In [1]:
# !pip install pyDecision # Uncomment this line for installation
import numpy as np
from pyDecision.algorithm import electre_iii

In [39]:
# =========================
# 1. Matriz de decisión
# =========================
matrix = np.array([
    [80, 50, 120],   # A
    [75, 40, 150],   # B
    [85, 55, 110]    # C
])

# =========================
# 2. Pesos (deben sumar 1)
# =========================
weights = np.array([0.4, 0.35, 0.25])

# =========================
# 3. Tipos de criterio
# g1 = Beneficio (max)
# g2 = Costo (min)
# g3 = Latencia (min)
types = ['max', 'min', 'min']
# =========================
#types = np.array([1, -1, -1])

# =========================
# 4. Umbrales de ELECTRE III
# q = indiferencia
# p = preferencia
# v = veto
# =========================
q = np.array([2, 3, 5])   # Indiferencia
p = np.array([6, 8, 15])     # Preferencia
v = np.array([15, 20, 40])     # Veto

# =========================
# 5. Ejecutar ELECTRE III
# =========================
# Orden correcto:
result = electre_iii(matrix, p, q, v, weights, graph=False)

# =========================
# 6. Mostrar resultado
# =========================
print("Resultado:")
print(result)

Resultado:
(array([[1.   , 0.75 , 0.56 ],
       [0.35 , 1.   , 0.25 ],
       [0.875, 0.75 , 1.   ]]), array([[0.    , 0.75  , 0.56  ],
       [0.35  , 0.    , 0.1029],
       [0.875 , 0.    , 0.    ]]), ['a3', 'a1', 'a2'], ['a3', 'a1', 'a2'], ['a1', 'a2', 'a3'], array([['-', 'P+', 'P-'],
       ['P-', '-', 'P-'],
       ['P+', 'P+', '-']], dtype='<U25'))


In [35]:
global_conc, credibility, rank_D, rank_A, rank_M, rank_P = result

print("Ranking descendente (D):", rank_D)
print("Ranking ascendente (A):", rank_A)
print("Ranking final recomendado:", rank_D)  # si D y A coinciden


Ranking descendente (D): ['a3', 'a1', 'a2']
Ranking ascendente (A): ['a3', 'a1', 'a2']
Ranking final recomendado: ['a3', 'a1', 'a2']


In [36]:
ranking_final = rank_D if rank_D == rank_A else rank_M


In [37]:
ranking_final

['a3', 'a1', 'a2']

In [38]:
print("Matriz de credibilidad:")
print(credibility)

Matriz de credibilidad:
[[0.     0.75   0.56  ]
 [0.35   0.     0.1029]
 [0.875  0.     0.    ]]


### Caso práctico ###

1) Datos del problema
- Alternativas: *A, B, C*
- Criterios (mezcla max/min), pesos
  - $g_1$: Beneficio (Max), $w_1$ = 0.4
  - $g_2$: Costo (Min),     $w_2$ = 0.35
  - $g_3$: Latencia (Min),  $w_3$ = 0.25


In [7]:
import numpy as np

# =========================
# 1. Datos del problema
# =========================
# Tabla de desempeño de las alternativas (A, B, C) para los criterios (g1, g2, g3)
matrix = np.array([
    [80, 50, 120],   # A
    [75, 40, 150],   # B
    [85, 55, 110]    # C
])
# =========================

# 2. Pesos de los criterios (deben sumar 1)
# =========================
weights = np.array([0.4, 0.35, 0.25])
# =========================
# 3. Tipos de criterio
# g1 = Beneficio (max)  
# g2 = Costo (min) 
# g3 = Latencia (min) 
# =========================
types = np.array([1, -1, -1])

# =========================
# 4. Umbrales de ELECTRE III
# q = indiferencia
# p = preferencia
# v = veto
# =========================
q = np.array([2, 3, 5])   # Indiferencia
p = np.array([6, 8, 15])     # Preferencia
v = np.array([15, 20, 40])     # Veto

In [29]:
alts = ["A", "B", "C"]
crits = ["g1", "g2", "g3"]

# =========================
# 2) Paso: diferencias d_j(a,b) (positivo => a mejor que b)
#    - MAX: d = g(a) - g(b)
#    - MIN: d = g(b) - g(a)
# =========================
def differences_by_criterion(matrix: np.ndarray, types: np.ndarray) -> np.ndarray:
    """
    Devuelve D con shape (m, n, n):
      D[j, i, k] = d_j(alt_i, alt_k)
    donde d>0 significa "alt_i es mejor que alt_k" en el criterio j.
    """
    n, m = matrix.shape
    D = np.zeros((m, n, n), dtype=float)

    for j in range(m):
        g = matrix[:, j]  # valores del criterio j para cada alternativa
        if types[j] == 1:     # MAX
            D[j] = g[:, None] - g[None, :]
        else:                 # MIN
            D[j] = g[None, :] - g[:, None]
    return D

d = differences_by_criterion(matrix, types)

# Imprime las matrices d(gj)
for j, name in enumerate(crits):
    print(f"\n--- d({name}) ---")
    print("     " + "  ".join(alts))
    for i, a in enumerate(alts):
        row = "  ".join(f"{D[j,i,k]:6.1f}" for k in range(len(alts)))
        print(f"{a}  {row}")



--- d(g1) ---
     A  B  C
A     0.0     5.0    -5.0
B    -5.0     0.0   -10.0
C     5.0    10.0     0.0

--- d(g2) ---
     A  B  C
A     0.0   -10.0     5.0
B    10.0     0.0    15.0
C    -5.0   -15.0     0.0

--- d(g3) ---
     A  B  C
A     0.0    30.0   -10.0
B   -30.0     0.0   -40.0
C    10.0    40.0     0.0


In [26]:
# =========================
# 3) Paso: concordancia parcial c_j(a,b) (una matriz n×n por criterio)
#    Usando d definido arriba:
#      c = 1              si d >= -q
#      c = 0              si d <= -p
#      c = (d + p)/(p-q)  si -p < d < -q
# =========================
def partial_concordance(D: np.ndarray, q: np.ndarray, p: np.ndarray) -> np.ndarray:
    """
    Devuelve Cj con shape (m, n, n):
      Cj[j,i,k] = c_j(alt_i, alt_k)
    """
    m, n, _ = D.shape
    Cj = np.zeros_like(D)

    for j in range(m):
        dj = D[j]
        qj, pj = q[j], p[j]

        Cj[j] = np.where(
            dj >= -qj, 1.0,
            np.where(dj <= -pj, 0.0, (dj + pj) / (pj - qj))
        )

        # Convención: diagonal = 1 (misma alternativa)
        np.fill_diagonal(Cj[j], 1.0)

    return Cj

Cj = partial_concordance(D, q, p)

# Imprime c(gj)
for j, name in enumerate(crits):
    print(f"\n--- c({name}) (concordancia parcial) ---")
    print("     " + "  ".join(alts))
    for i, a in enumerate(alts):
        row = "  ".join(f"{Cj[j,i,k]:6.3f}" for k in range(len(alts)))
        print(f"{a}  {row}")


--- c(g1) (concordancia parcial) ---
     A  B  C
A   1.000   1.000   0.250
B   0.250   1.000   0.000
C   1.000   1.000   1.000

--- c(g2) (concordancia parcial) ---
     A  B  C
A   1.000   0.000   1.000
B   1.000   1.000   1.000
C   0.600   0.000   1.000

--- c(g3) (concordancia parcial) ---
     A  B  C
A   1.000   1.000   0.500
B   0.000   1.000   0.000
C   1.000   1.000   1.000


In [27]:
# =========================
# (Opcional) 4) Concordancia global C(a,b)
#    C = sum_j w_j * c_j(a,b)
# =========================
C_global = np.tensordot(weights, Cj, axes=(0, 0))  # (n,n)
np.fill_diagonal(C_global, 0.0)  # a veces se pone 0 en diagonal

print("\n--- C (concordancia global) ---")
print("     " + "  ".join(alts))
for i, a in enumerate(alts):
    row = "  ".join(f"{C_global[i,k]:6.3f}" for k in range(len(alts)))
    print(f"{a}  {row}")


--- C (concordancia global) ---
     A  B  C
A   0.000   0.650   0.575
B   0.450   0.000   0.350
C   0.860   0.650   0.000


In [30]:
# =========================
# 5) Discordancia parcial D_j(a,b)
# =========================
def partial_discordance(d, p, v):
    m, n, _ = d.shape
    Dj = np.zeros_like(d)

    for j in range(m):
        dj = d[j]
        pj, vj = p[j], v[j]

        Dj[j] = np.where(
            dj >= -pj, 0.0,
            np.where(dj <= -vj, 1.0, (-pj - dj) / (vj - pj))
        )
        np.fill_diagonal(Dj[j], 0.0)

    return Dj

D_partial = partial_discordance(d, p, v)


In [31]:
# =========================
# 6) Matriz de credibilidad S(a,b)
# =========================
def credibility_matrix(C, D_partial):
    m, n, _ = D_partial.shape
    S = C.copy()

    for i in range(n):
        for j in range(n):
            if i == j:
                S[i, j] = 0
                continue

            prod = 1.0
            for k in range(m):
                if D_partial[k, i, j] > C[i, j]:
                    prod *= (1 - D_partial[k, i, j]) / (1 - C[i, j])

            S[i, j] = C[i, j] * prod

    return S

S = credibility_matrix(C_global, D_partial)

In [32]:
# =========================
# 7) Impresión ordenada
# =========================
np.set_printoptions(precision=4, suppress=True)

print("\n=== Concordancia global C ===")
print(C)

print("\n=== Matriz de credibilidad S ===")
print(S)


=== Concordancia global C ===
[[0.    0.65  0.575]
 [0.45  0.    0.35 ]
 [0.86  0.65  0.   ]]

=== Matriz de credibilidad S ===
[[0.     0.65   0.575 ]
 [0.3273 0.     0.    ]
 [0.86   0.65   0.    ]]


In [8]:
X = matrix.copy()
X[:, types == -1] *= -1   # convierte costo→beneficio

# X ahora está en escala "más grande = mejor" para todo
print("Matriz orientada a beneficio:\n", X)


Matriz orientada a beneficio:
 [[  80  -50 -120]
 [  75  -40 -150]
 [  85  -55 -110]]


In [9]:
# shape: (n_alt, n_alt, n_crit)
Delta = X[np.newaxis, :, :] - X[:, np.newaxis, :]

# Delta[i,j,k] = X[j,k] - X[i,k]
print("Delta (i,j,k):\n", Delta)


Delta (i,j,k):
 [[[  0   0   0]
  [ -5  10 -30]
  [  5  -5  10]]

 [[  5 -10  30]
  [  0   0   0]
  [ 10 -15  40]]

 [[ -5   5 -10]
  [-10  15 -40]
  [  0   0   0]]]


Concordancia parcial $c_k(i,j)$

In [10]:
# broadcasting: q,p a (1,1,m)
Q = q[np.newaxis, np.newaxis, :]
P = p[np.newaxis, np.newaxis, :]

c = np.where(
    Delta <= Q, 1.0,
    np.where(Delta >= P, 0.0, (P - Delta) / (P - Q))
)

# Por definición, c[i,i,k] suele ser 1 (misma alternativa)
print("Concordancia parcial c_k:\n", c)


Concordancia parcial c_k:
 [[[1.   1.   1.  ]
  [1.   0.   1.  ]
  [0.25 1.   0.5 ]]

 [[0.25 1.   0.  ]
  [1.   1.   1.  ]
  [0.   1.   0.  ]]

 [[1.   0.6  1.  ]
  [1.   0.   1.  ]
  [1.   1.   1.  ]]]


Concordancia global C(i,j)

In [11]:
w = weights[np.newaxis, np.newaxis, :]
C = np.sum(w * c, axis=2)

# muchas implementaciones ponen la diagonal en 0 (no se compara a sí misma)
np.fill_diagonal(C, 0.0)

print("Concordancia global C:\n", C)


Concordancia global C:
 [[0.    0.65  0.575]
 [0.45  0.    0.35 ]
 [0.86  0.65  0.   ]]


Discordancia D_k(i,j)

In [12]:
V = v[np.newaxis, np.newaxis, :]

D = np.where(
    Delta <= P, 0.0,
    np.where(Delta >= V, 1.0, (Delta - P) / (V - P))
)

np.fill_diagonal(D[:, :, 0], 0.0)
np.fill_diagonal(D[:, :, 1], 0.0)
np.fill_diagonal(D[:, :, 2], 0.0)

print("Discordancia por criterio D_k:\n", D)


Discordancia por criterio D_k:
 [[[0.         0.         0.        ]
  [0.         0.16666667 0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.6       ]
  [0.         0.         0.        ]
  [0.44444444 0.         1.        ]]

 [[0.         0.         0.        ]
  [0.         0.58333333 0.        ]
  [0.         0.         0.        ]]]


Matriz de credibilidad $S(i,j)$

In [13]:
S = C.copy()
den = (1.0 - C)
den_safe = np.where(den == 0, 1e-12, den)  # evita división por cero

mask = D > C[:, :, np.newaxis]   # criterios donde hay discordancia fuerte

# factor = (1-D)/(1-C) solo donde D>C, si no factor=1
factor = np.where(mask, (1.0 - D) / den_safe[:, :, np.newaxis], 1.0)

S = C * np.prod(factor, axis=2)
np.fill_diagonal(S, 0.0)

print("Credibilidad S:\n", S)


Credibilidad S:
 [[0.         0.65       0.575     ]
 [0.32727273 0.         0.        ]
 [0.86       0.65       0.        ]]


In [14]:
import numpy as np

matrix = np.array([
    [80, 50, 120],   # A
    [75, 40, 150],   # B
    [85, 55, 110]    # C
], dtype=float)

weights = np.array([0.4, 0.35, 0.25], dtype=float)
types   = np.array([1, -1, -1])          # 1=beneficio, -1=costo
q = np.array([2, 3, 5], dtype=float)
p = np.array([6, 8, 15], dtype=float)
v = np.array([15, 20, 40], dtype=float)

X = matrix.copy()
X[:, types == -1] *= -1   # convierte costo→beneficio

# X ahora está en escala "más grande = mejor" para todo
print("Matriz orientada a beneficio:\n", X)




Matriz orientada a beneficio:
 [[  80.  -50. -120.]
 [  75.  -40. -150.]
 [  85.  -55. -110.]]


In [17]:
# shape: (n_alt, n_alt, n_crit)
Delta = X[np.newaxis, :, :] - X[:, np.newaxis, :]

# Delta[i,j,k] = X[j,k] - X[i,k]
print("Delta (i,j,k):\n", Delta)


Delta (i,j,k):
 [[[  0.   0.   0.]
  [ -5.  10. -30.]
  [  5.  -5.  10.]]

 [[  5. -10.  30.]
  [  0.   0.   0.]
  [ 10. -15.  40.]]

 [[ -5.   5. -10.]
  [-10.  15. -40.]
  [  0.   0.   0.]]]


In [18]:
# broadcasting: q,p a (1,1,m)
Q = q[np.newaxis, np.newaxis, :]
P = p[np.newaxis, np.newaxis, :]

c = np.where(
    Delta <= Q, 1.0,
    np.where(Delta >= P, 0.0, (P - Delta) / (P - Q))
)

# Por definición, c[i,i,k] suele ser 1 (misma alternativa)
print("Concordancia parcial c_k:\n", c)


Concordancia parcial c_k:
 [[[1.   1.   1.  ]
  [1.   0.   1.  ]
  [0.25 1.   0.5 ]]

 [[0.25 1.   0.  ]
  [1.   1.   1.  ]
  [0.   1.   0.  ]]

 [[1.   0.6  1.  ]
  [1.   0.   1.  ]
  [1.   1.   1.  ]]]


In [19]:
w = weights[np.newaxis, np.newaxis, :]
C = np.sum(w * c, axis=2)

# muchas implementaciones ponen la diagonal en 0 (no se compara a sí misma)
np.fill_diagonal(C, 0.0)

print("Concordancia global C:\n", C)


Concordancia global C:
 [[0.    0.65  0.575]
 [0.45  0.    0.35 ]
 [0.86  0.65  0.   ]]


In [20]:
V = v[np.newaxis, np.newaxis, :]

D = np.where(
    Delta <= P, 0.0,
    np.where(Delta >= V, 1.0, (Delta - P) / (V - P))
)

np.fill_diagonal(D[:, :, 0], 0.0)
np.fill_diagonal(D[:, :, 1], 0.0)
np.fill_diagonal(D[:, :, 2], 0.0)

print("Discordancia por criterio D_k:\n", D)


Discordancia por criterio D_k:
 [[[0.         0.         0.        ]
  [0.         0.16666667 0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.6       ]
  [0.         0.         0.        ]
  [0.44444444 0.         1.        ]]

 [[0.         0.         0.        ]
  [0.         0.58333333 0.        ]
  [0.         0.         0.        ]]]


In [21]:
S = C.copy()
den = (1.0 - C)
den_safe = np.where(den == 0, 1e-12, den)  # evita división por cero

mask = D > C[:, :, np.newaxis]   # criterios donde hay discordancia fuerte

# factor = (1-D)/(1-C) solo donde D>C, si no factor=1
factor = np.where(mask, (1.0 - D) / den_safe[:, :, np.newaxis], 1.0)

S = C * np.prod(factor, axis=2)
np.fill_diagonal(S, 0.0)

print("Credibilidad S:\n", S)


Credibilidad S:
 [[0.         0.65       0.575     ]
 [0.32727273 0.         0.        ]
 [0.86       0.65       0.        ]]
